### Importing Libraries
The first step in any Python script or Jupyter notebook is to import the necessary libraries. Here, we are importing libraries that will be used for stock data retrieval, risk portfolio analysis, time management, threading, and interaction with the Interactive Brokers API.

In [2]:
import time
import threading
import numpy as np
import pandas as pd
import datetime
from openbb_terminal.sdk import openbb
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import local_hist_db3 as adb

Current time in NZ: 2023-12-28 15:53:54 NZDT
Current time in NY: 2023-12-27 21:53:54 EST


In [3]:
# Set the values we'll use in the trading app
today = pd.Timestamp.today().strftime("%Y-%m-%d")
top_N = 5

### Universe Selection
In this section, we are using the openbb_terminal.sdk library to screen for stocks that have recently hit new highs. This is as much art as science. Use your skill and experience to select your investment universe. Ideally a group of stocks you think are showing signs of momentum.

In [4]:
new_highs = openbb.stocks.screener.screener_data("new_high")

### Data Filtering
After retrieving the stock data, we filter it based on certain criteria. Here, we are filtering for stocks with a price greater than $15 and that are based in the USA.

In [5]:
port_data = new_highs[
    (new_highs.Price > 15) &
    (new_highs.Country == "USA")
]

### Data Preparation
Once we have our filtered list of stocks, we extract their ticker symbols and then use these symbols to download historical stock price data. This data will be used for further analysis.

In [6]:
symbols = port_data.Ticker.tolist()

In [7]:
# symbols

In [8]:
for sym in port_data.Ticker.tolist():
    adb.get_data(sym)

CYTK Data up to date 2020-12-23 to 2023-12-27
MARA Data up to date 2020-12-23 to 2023-12-27
MSTR Data up to date 2020-12-23 to 2023-12-27
ASPN Data up to date 2020-12-23 to 2023-12-27
COIN Data up to date 2021-04-14 to 2023-12-27
REFI Data up to date 2021-12-08 to 2023-12-27
BBIO Data up to date 2020-12-23 to 2023-12-27
CBAY Data up to date 2020-12-23 to 2023-12-27
ACU Data up to date 2020-12-23 to 2023-12-27
LMNR Data up to date 2020-12-23 to 2023-12-27
AGM Data up to date 2020-12-23 to 2023-12-27
ITCI Data up to date 2020-12-23 to 2023-12-27
MGRC Data up to date 2020-12-23 to 2023-12-27


RCKT Data up to date 2020-12-23 to 2023-12-27
CVRX Data up to date 2021-06-30 to 2023-12-27
STRL Data up to date 2020-12-23 to 2023-12-27
GFF Data up to date 2020-12-23 to 2023-12-27
MMI Data up to date 2020-12-23 to 2023-12-27
CUBI Data up to date 2020-12-23 to 2023-12-27
WD Data up to date 2020-12-23 to 2023-12-27
BHVN Data up to date 2022-09-23 to 2023-12-27
FRPT Data up to date 2020-12-23 to 2023-12-27
RYTM Data up to date 2020-12-23 to 2023-12-27
SWTX Data up to date 2020-12-23 to 2023-12-27
KD Data up to date 2021-10-22 to 2023-12-27
LMB Data up to date 2020-12-23 to 2023-12-27
HLNE Data up to date 2020-12-23 to 2023-12-27
COKE Data up to date 2020-12-23 to 2023-12-27
FSTR Data up to date 2020-12-23 to 2023-12-27
ARVN Data up to date 2020-12-23 to 2023-12-27
NWSA Data up to date 2020-12-23 to 2023-12-27
NWS Data up to date 2020-12-23 to 2023-12-27
RXST Data up to date 2021-07-30 to 2023-12-27
MHO Data up to date 2020-12-23 to 2023-12-27
AMD Data up to date 2020-12-23 to 2023-12-2

In [9]:
symbol_short_list = []
for sym in adb.ac['DAILY'].list_symbols():
    size = adb.ac['DAILY'].read(sym).data.shape[0]
    if size > 2 * 12 * 21:
        symbol_short_list.append(sym)

print(symbol_short_list)

['KD', 'DELL', 'ASAN', 'TWLO', 'MDC', 'HBB', 'LMB', 'MMI', 'DSGR', 'CRVL', 'LOB', 'AFRM', 'BBSI', 'NWSA', 'DOCU', 'FICO', 'WMS', 'STC', 'BPMC', 'INSM', 'FTNT', 'BKNG', 'CCS', 'BCC', 'SPXC', 'GTLS', 'REFI', 'PCVX', 'DIS', 'INTC', 'EVR', 'RKT', 'UHAL', 'BAND', 'ANDE', 'CABA', 'JLL', 'COIN', 'MLI', 'UDMY', 'AMD', 'SCSC', 'MGNI', 'RAMP', 'TSLA', 'MGRC', 'GHM', 'MHO', 'CVS', 'HLNE', 'NEM', 'FSTR', 'ODP', 'TDW', 'TXRH', 'RYTM', 'ANET', 'MTCH', 'FSLY', 'ZS', 'REGN', 'MCD', 'KOP', 'CRSP', 'META', 'PJT', 'CENT', 'ICE', 'MS', 'SWTX', 'KBH', 'PATH', 'LMNR', 'U', 'MTN', 'CVRX', 'BHP', 'CARG', 'BFST', 'CUBI', 'LMND', 'CHCO', 'FNF', 'MAT', 'CFLT', 'KO', 'MEDP', 'PINS', 'NPO', 'BWMN', 'ACU', 'K', 'CRM', 'SHOP', 'WDAY', 'MSTR', 'CYTK', 'GOOG', 'MSFT', 'ESTC', 'STRL', 'KELYA', 'SKY', 'ZEUS', 'RCKT', 'ML', 'HUBS', 'GOLF', 'NCNO', 'ASML', 'ARVN', 'GDRX', 'FRPT', 'CRWD', 'NWS', 'NFLX', 'ALG', 'HAS', 'COKE', 'BBIO', 'JELD', 'ITCI', 'IBOC', 'NOW', 'DLX', 'PATK', 'FVRR', 'SKYW', 'MASI', 'EAT', 'BCPC', 'SSD',

In [24]:
# The function first computes the percentage change in the 
# closing prices over the most recent 126 trading days, 
# which is used later for normalization. The momentum 
# score is then calculated as the difference between 
# the long-term return (the percentage change in closing 
# price over the past 252 days) and the short-term return 
# (the percentage change in closing price over the past 21
# days). This score is normalized by dividing it by the 
# standard deviation of the returns over the past 126 days, 
# providing a standardized measure of momentum that accounts 
# for recent volatility. 
def momentum(close):
    returns = close.pct_change()[-126:]
    return(
        (close[-21] - close[-252]) / close[-252] 
        - (close[-1] - close[-21]) / close[-21]
    ) / np.std(returns)

In [29]:
q = adb.QueryBuilder()
q = q.apply('momentum', momentum(q['close']))


AttributeError: 'ExpressionNode' object has no attribute 'pct_change'

In [49]:
q = adb.QueryBuilder()
# q = q.apply('momentum', q['close'] * 2)
q = q.date_range((pd.Timestamp(adb.today), pd.Timestamp(adb.today)))


In [50]:
adb.ac['DAILY'].read_batch(["KD","DELL"], query_builder=q )

[VersionedItem(symbol='KD', library='DAILY', data=<class 'pandas.core.frame.DataFrame'>, version=0, metadata=None, host='LMDB(path=c:\\data\\arcticdb\\obb3)'),
 VersionedItem(symbol='DELL', library='DAILY', data=<class 'pandas.core.frame.DataFrame'>, version=0, metadata=None, host='LMDB(path=c:\\data\\arcticdb\\obb3)')]

In [7]:
# Using an arbitrary list of stocks for demonstration purposes. This
# Overrides the stock universe created above.
symbols = ["NEM", "RGLD", "SSRM", "CDE", "LLY", "UNH", "JNJ", "MRK"]
stocks = []
for symbol in symbols:
    df = (
        openbb
        .stocks
        .load(
            symbol,
            start_date="2015-01-01", 
            end_date=today,
            verbose=False
        )
        .drop(["Close", "Dividends", "Stock Splits"], axis=1)
    )
    df["symbol"] = symbol
    stocks.append(df)

prices = pd.concat(stocks)
prices.columns = ["open", "high", "low", "close", "volume", "symbol"]

In [8]:
prices

,open,high,low,close,volume,symbol
date,,,,,,
2015-01-02,15.178799,15.781068,15.138106,15.740375,6533500,NEM
2015-01-05,15.878728,15.951977,15.349709,15.838034,7508100,NEM
2015-01-06,15.992674,16.814690,15.968259,16.497278,13198100,NEM
2015-01-07,16.310085,16.700746,16.065922,16.350779,8172100,NEM
2015-01-08,16.358915,16.586800,16.188001,16.261250,7803400,NEM
...,...,...,...,...,...,...
2023-12-20,107.139999,107.139999,105.239998,105.360001,8441600,MRK
2023-12-21,105.790001,106.610001,105.339996,106.389999,5919400,MRK
2023-12-22,107.150002,108.059998,106.839996,107.699997,6028100,MRK


### Factor Engineering
With the historical stock price data in hand, we can now proceed to engineer our momentum factor.

In [39]:
# Check to make sure we have at least 2 years worth of data
nobs = prices.groupby("symbol").size()
mask = nobs[nobs > 2 * 12 * 21].index
prices = prices[prices.symbol.isin(mask)]

In [40]:
prices

,open,high,low,close,volume,symbol
date,,,,,,
2019-06-28,18.500000,24.440001,17.260000,20.020000,2734600,KRTX
2019-07-01,22.389999,22.389999,19.833000,21.000000,381000,KRTX
2019-07-02,21.219999,22.530001,21.180000,21.700001,105300,KRTX
2019-07-03,21.610001,25.200001,21.610001,24.450001,191800,KRTX
2019-07-05,25.000000,29.000000,22.857000,27.350000,330900,KRTX
...,...,...,...,...,...,...
2023-12-18,123.190002,123.849998,122.419998,123.559998,1999400,ICE
2023-12-19,123.559998,123.959999,123.070000,123.769997,1785000,ICE
2023-12-20,123.570000,124.050003,122.309998,122.360001,2350500,ICE


In [41]:
# Arrange the DataFrame to have a symbol and date index
prices = (
    prices
    .set_index("symbol", append=True)
    .reorder_levels(["symbol", "date"])
).drop_duplicates()

In [42]:
prices

open        high         low       close   volume
symbol date                                                               
KRTX   2019-06-28   18.500000   24.440001   17.260000   20.020000  2734600
       2019-07-01   22.389999   22.389999   19.833000   21.000000   381000
       2019-07-02   21.219999   22.530001   21.180000   21.700001   105300
       2019-07-03   21.610001   25.200001   21.610001   24.450001   191800
       2019-07-05   25.000000   29.000000   22.857000   27.350000   330900
...                       ...         ...         ...         ...      ...
ICE    2023-12-18  123.190002  123.849998  122.419998  123.559998  1999400
       2023-12-19  123.559998  123.959999  123.070000  123.769997  1785000
       2023-12-20  123.570000  124.050003  122.309998  122.360001  2350500
       2023-12-21  122.720001  124.330002  122.650002  124.230003  1895400
       2023-12-22  124.910004  125.910004  124.300003  125.720001  2231719

[224481 rows x 5 columns]

In [44]:
# Apply the momentum factor to a 252 trading day rolling
# window to generate the momentum factor
df = (
    prices
    .groupby(
        "symbol", 
        group_keys=False
    )
    .rolling(252)
    .close
    .apply(momentum)
)

In [45]:
# Pandas adds an extra symbol index, so remove it
df.index = df.index.droplevel(0)

In [46]:
# Add our momentum factor values to our original DataFrame
# and drop any NA values
prices["momentum"] = df
prices.dropna(inplace=True)

In [47]:
prices

open        high         low       close   volume  \
symbol date                                                                  
KRTX   2020-06-26  117.489998  118.209999  108.019997  108.389999   968600   
       2020-06-29  108.940002  110.870003  104.410004  106.260002   204100   
       2020-06-30  105.889999  112.269997  104.540001  111.459999   257600   
       2020-07-01  111.230003  111.980003  104.050003  106.459999   192300   
       2020-07-02  107.760002  111.150002  105.769997  109.739998   156200   
...                       ...         ...         ...         ...      ...   
ICE    2023-12-18  123.190002  123.849998  122.419998  123.559998  1999400   
       2023-12-19  123.559998  123.959999  123.070000  123.769997  1785000   
       2023-12-20  123.570000  124.050003  122.309998  122.360001  2350500   
       2023-12-21  122.720001  124.330002  122.650002  124.230003  1895400   
       2023-12-22  124.910004  125.910004  124.300003  125.720001  2231719   

                    momentum  
symbol date                   
KRTX   2020-06-26  64.734511  
       2020-06-29  58.820253  
       2020-06-30  56.456077  
       2020-07-01  53.663311  
       2020-07-02  45.244361  
...                      ...  
ICE    2023-12-18   0.537172  
       2023-12-19   2.325215  
       2023-12-20   5.876553  
       2023-12-21   3.572921  
       2023-12-22   2.546648  

[197624 rows x 6 columns]

In [48]:
# Rank the momentum factor for each stock, every day
prices["factor_rank"] = (
    prices
    .groupby(level=[1])
    .momentum
    .rank(ascending=False)
)

In [49]:
prices

open        high         low       close   volume  \
symbol date                                                                  
KRTX   2020-06-26  117.489998  118.209999  108.019997  108.389999   968600   
       2020-06-29  108.940002  110.870003  104.410004  106.260002   204100   
       2020-06-30  105.889999  112.269997  104.540001  111.459999   257600   
       2020-07-01  111.230003  111.980003  104.050003  106.459999   192300   
       2020-07-02  107.760002  111.150002  105.769997  109.739998   156200   
...                       ...         ...         ...         ...      ...   
ICE    2023-12-18  123.190002  123.849998  122.419998  123.559998  1999400   
       2023-12-19  123.559998  123.959999  123.070000  123.769997  1785000   
       2023-12-20  123.570000  124.050003  122.309998  122.360001  2350500   
       2023-12-21  122.720001  124.330002  122.650002  124.230003  1895400   
       2023-12-22  124.910004  125.910004  124.300003  125.720001  2231719   

                    momentum  factor_rank  
symbol date                                
KRTX   2020-06-26  64.734511          1.0  
       2020-06-29  58.820253          1.0  
       2020-06-30  56.456077          1.0  
       2020-07-01  53.663311          1.0  
       2020-07-02  45.244361          1.0  
...                      ...          ...  
ICE    2023-12-18   0.537172         77.0  
       2023-12-19   2.325215         75.0  
       2023-12-20   5.876553         69.0  
       2023-12-21   3.572921         66.0  
       2023-12-22   2.546648         69.0  

[197624 rows x 7 columns]

In [59]:
prices.reset_index().groupby('symbol').apply(lambda x: x.loc[x['date'].idxmax()])

,symbol,date,open,high,low,close,volume,momentum,factor_rank
symbol,,,,,,,,,
ACA,ACA,2023-12-22,83.000000,83.764999,82.930000,83.419998,123732,13.627131,37.0
ALPN,ALPN,2023-12-22,20.180000,20.910000,19.150000,19.420000,807077,13.241828,39.0
ALTR,ALTR,2023-12-22,77.940002,84.577103,77.000000,83.290001,1304394,19.653629,29.0
ALXO,ALXO,2023-12-22,13.400000,15.480000,13.400000,15.380000,928223,-11.145825,100.0
ANSS,ANSS,2023-12-22,363.010010,363.859985,328.000000,357.980011,5146063,2.385641,70.0
...,...,...,...,...,...,...,...,...,...
UVV,UVV,2023-12-22,65.199997,65.889999,64.769997,65.120003,84968,-5.302841,84.0
WDC,WDC,2023-12-22,52.500000,52.880001,52.240002,52.660000,2925978,18.004556,30.0
WEYS,WEYS,2023-12-22,32.700001,32.985001,32.520000,32.740002,14706,5.304869,64.0


In [60]:
# Generate the portfolio we want to trade based on the Top
# N stocks we want to trade
stocks_to_trade = (
    prices.reset_index()
    .groupby('symbol')
    .apply(lambda x: x.loc[x['date'].idxmax()])
    .sort_values("factor_rank")
    .head(top_N)
)

In [61]:
stocks_to_trade

,symbol,date,open,high,low,close,volume,momentum,factor_rank
symbol,,,,,,,,,
FTAI,FTAI,2023-12-22,45.869999,47.150002,45.869999,46.990002,485542,70.936851,1.0
LPG,LPG,2023-12-22,45.910000,47.189899,45.720001,46.439999,550174,56.981019,2.0
GE,GE,2023-12-22,127.389999,128.020004,126.180000,126.690002,2979442,55.592215,3.0
MSTR,MSTR,2023-12-22,581.960022,622.580017,578.700012,619.239990,1247698,53.753029,4.0
CBAY,CBAY,2023-12-22,22.280001,23.490000,22.230000,23.090000,1431646,51.244111,5.0


### Compute the number of shares to purchase
Compute the number of shares to purchase based on the optimized weights

In [26]:
port_val = 60_000

In [27]:
stocks_to_trade

NameError: name 'stocks_to_trade' is not defined

In [22]:
stocks_to_trade["invest_amt"] = (1 / top_N) * port_val

In [23]:
stocks_to_trade

,open,high,low,close,volume,momentum,factor_rank,invest_amt
symbol,,,,,,,,
LLY,572.099976,573.369995,562.000977,570.309998,2891309,32.231306,1.0,2000.0
UNH,530.000000,533.500000,523.000000,528.104980,3449131,4.372983,2.0,2000.0
RGLD,120.540001,121.489998,118.680000,118.720001,168454,2.133716,3.0,2000.0
SSRM,10.590000,10.930000,10.540000,10.830000,2188299,-4.862041,4.0,2000.0
MRK,104.889999,105.959999,104.260002,105.830002,5461533,-8.586806,5.0,2000.0


In [24]:
stocks_to_trade["shares"] = (stocks_to_trade.invest_amt / stocks_to_trade.close).astype(int)

In [25]:
(stocks_to_trade.shares * stocks_to_trade.close).sum()

9092.42497253418

### Execute the orders on IB
Set up the IB app, contracts, and order submission. Connect to the app and execute orders through the API.

In [5]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
    
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextOrderId = orderId

In [59]:
def stock_contract(symbol, secType="STK", exchange="SMART", currency="USD"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency

    return contract

def submit_order(contract, direction, qty=100, orderType="MKT", transmit=True):
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = orderType
    order.transmit = transmit
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    # submit order
    app.placeOrder(app.nextOrderId, contract, order)
    app.nextOrderId += 1

In [8]:
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7497, 123)
app.nextOrderId = None

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

while True:
    if isinstance(app.nextOrderId, int):
        print("Connected")
        break
    else:
        print("Waiting")
        time.sleep(1)

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Waiting
Connected


In [60]:
for row in stocks_to_trade.itertuples():
    contract = stock_contract(row.Index)
    submit_order(contract, direction="BUY", qty=row.shares)